In [1]:
import numpy as np

## ３層ニューラルネットワークの実装(p.65)

In [2]:
def identity_function(x): # 恒等関数
    return x

def sigmoid(x): # シグモイド関数
    return 1 / (1 + np.exp(-x))

def init_network(): # 重みとバイアスの初期化
    network = {} # ディクショナリ型
    network['W1'] = np.array([[0.1, 0.3, 0.5], [0.2, 0.4, 0.6]])
    network['b1'] = np.array([0.1, 0.2, 0.3])
    network['W2'] = np.array([[0.1, 0.4], [0.2, 0.5], [0.3, 0.6]])
    network['b2'] = np.array([0.1, 0.2])
    network['W3'] = np.array([[0.1,0.3],[0.2, 0.4]])
    network['b3'] = np.array([0.1, 0.2])
    return network

def forward(network, x): # 入力から出力方向への伝達処理
    W1, W2, W3 = network['W1'], network['W2'], network['W3']
    b1, b2, b3 = network['b1'], network['b2'], network['b3']

    a1 = np.dot(x, W1) + b1
    z1 = sigmoid(a1)
    a2 = np.dot(z1, W2) + b2
    z2 = sigmoid(a2)
    a3 = np.dot(z2, W3) + b3
    y = identity_function(a3)
    return y

In [3]:
network = init_network()
print(network)

{'W1': array([[0.1, 0.3, 0.5],
       [0.2, 0.4, 0.6]]), 'b1': array([0.1, 0.2, 0.3]), 'W2': array([[0.1, 0.4],
       [0.2, 0.5],
       [0.3, 0.6]]), 'b2': array([0.1, 0.2]), 'W3': array([[0.1, 0.3],
       [0.2, 0.4]]), 'b3': array([0.1, 0.2])}


In [4]:
x = np.array([1.0, 0.5])
y = forward(network, x)
print(y)

[0.31682708 0.69627909]


## ソフトマックス関数

In [5]:
# ソフトマックス関数
def softmax(x): 
    c = np.max(x)
    exp_a = np.exp(a - c) # オーバーフロー対策
    sum_exp_a = np.sum(exp_a)
    y  = exp_a/sum_exp_a
    return y

In [6]:
a = np.array([0.3, 2.9, 4.0])
y = softmax(a)
print(y)

[0.01821127 0.24519181 0.73659691]


In [7]:
np.sum(y)

1.0

## MNISTデータセットの読み込み

In [8]:
# MNISTデータセットの読み込み
import sys, os
sys.path.append(os.pardir)  # 親ディレクトリのファイルをインポートするための設定

from dataset.mnist import load_mnist
"""MNISTデータセットの読み込み

Parameters
初期値 normalize=True, flatten=True, one_hot_label=False
----------
normalize : 画像のピクセル値を0.0~1.0に正規化する
one_hot_label :
    one_hot_labelがTrueの場合、ラベルはone-hot配列として返す
    one-hot配列とは、たとえば[0,0,1,0,0,0,0,0,0,0]のような配列
flatten : 画像を一次元配列に平にするかどうか

Returns
-------
(訓練画像, 訓練ラベル), (テスト画像, テストラベル)
"""
(x_train, t_train), (x_test, t_test) = load_mnist(flatten=True, normalize=False)

print(x_train.shape)
print(t_train.shape)
print(x_test.shape)
print(t_test.shape)

(60000, 784)
(60000,)
(10000, 784)
(10000,)


## バッチ処理による処理時間の比較

In [9]:
# バッチ処理による処理時間の比較
# coding: utf-8
import sys, os
sys.path.append(os.pardir)  # 親ディレクトリのファイルをインポートするための設定
import pickle
from common.functions import sigmoid, softmax


def get_data():
    (x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, flatten=True, one_hot_label=False)
    return x_test, t_test


def init_network():
    with open("sample_weight.pkl", 'rb') as f:
        network = pickle.load(f)
    return network


def predict(network, x):
    W1, W2, W3 = network['W1'], network['W2'], network['W3']
    b1, b2, b3 = network['b1'], network['b2'], network['b3']

    a1 = np.dot(x, W1) + b1
    z1 = sigmoid(a1)
    a2 = np.dot(z1, W2) + b2
    z2 = sigmoid(a2)
    a3 = np.dot(z2, W3) + b3
    y = softmax(a3)
    return y

In [10]:
# 1. バッチ処理なし
%time
x, t = get_data()
network = init_network()
accuracy_cnt = 0
for i in range(len(x)):
    y = predict(network, x[i])
    p= np.argmax(y) # 最も確率の高い要素のインデックスを取得
    if p == t[i]: # 正解していた時
        accuracy_cnt += 1

print("Accuracy:" + str(float(accuracy_cnt) / len(x)))

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 6.91 µs
Accuracy:0.9352


In [11]:
# 2. バッチ処理あり
%time
x, t = get_data()
network = init_network()

batch_size = 100 # バッチの数
accuracy_cnt = 0

for i in range(0, len(x), batch_size):
    x_batch = x[i:i+batch_size]
    y_batch = predict(network, x_batch)
    p = np.argmax(y_batch, axis=1) # 1次元目の要素ごとに
    accuracy_cnt += np.sum(p == t[i:i+batch_size]) # Trueの個数を算出

print("Accuracy:" + str(float(accuracy_cnt) / len(x)))

CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 7.15 µs
Accuracy:0.9352
